<h1>Environmental Hashtag Positive/Negative Scoring System</h1>
<p><u><b>Objective:</b></u> Add a positive or negative comment to environmental comments from Twitter</p>

<h2><u>Import all the necessary libraries</u></h2>

In [37]:
# We add all the imports
import json # Import json to handle json files.
from nltk.corpus import wordnet  #Import wordnet from the NLTK
from nltk.corpus import sentiwordnet as swn # We import sentiwordnet

<h2><u>Create Data Cleaning Methods</u></h2>
<p>Created by <b>@Chhandosee Bhattacharya<b></p>
<p>References: <a href=https://medium.com/analytics-vidhya/working-with-twitter-data-b0aa5419532>link</a><p>

In [38]:
# Import libraries needed for data cleaning
import emoji
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

This function cleans the data and makes it ready for Word Embedding

In [39]:
def Data_Cleaning(lines):
    # Define a dictioary of emoticons
    dict_emo = { ':-)'  : b'\xf0\x9f\x98\x8a',
                      ':)'   : b'\xf0\x9f\x98\x8a',
                      '=)'   : b'\xf0\x9f\x98\x8a',  # Smile or happy
                     ':-D'  : b'\xf0\x9f\x98\x83',
                      ':D'   : b'\xf0\x9f\x98\x83',
                      '=D'   : b'\xf0\x9f\x98\x83',  # Big smile
                      '>:-(' : b'\xF0\x9F\x98\xA0',
                      '>:-o' : b'\xF0\x9F\x98\xA0'   # Angry face
                      }
    #Function converts emoticons to emoji
    def convert_emoticons(emoticons):
        emoticons=emoticons.replace('.',' ')
        emoticons=emoticons.replace(',',' ')
        for i in emoticons.split():
            if i in dict_emo.keys():
                word=dict_emo[i].decode('utf-8')
                emoticons=emoticons.replace(i,word)
        return emoticons

    #Function to convert emoji to word
    def convert_emoji_to_word(emo_converted_text):
        for i in emo_converted_text:
            if i in emoji.UNICODE_EMOJI:
                emo_word=str(emoji.demojize(i))
                rep_colon=emo_word.replace(':',' ')
                rep_dash=rep_colon.replace('_',' ')
                emo_converted_text=emo_converted_text.replace(i,rep_dash)
        return emo_converted_text
    # Function tokenizes the whole data, normalies it to lower case, removes 
    #stop words and stems the data
    def clean_data(text):
        tokens= word_tokenize(text)
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove all tokens that are not alphabetic
        words = [word for word in tokens if word.isalpha()]
        # filter out stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        porter = PorterStemmer()
        stemmed = [porter.stem(word) for word in words]
        return stemmed
	# Function to remove hyperlinks from a tweet
	# Created by Otoniel Campos
    def remove_hyperlinks(tweet):
	    result = re.sub(r"http\S+", "", tweet)
	    return result

    # Task of data Cleaning function starts here  
    text=str(lines)
    text=text.replace("ufeff"," ")
    text=text.replace("'","")
    emoticons_treated=convert_emoticons(text)
    emo_treated_lines=convert_emoji_to_word(emoticons_treated)
    removed_hyperlinks=remove_hyperlinks(emo_treated_lines)
    cleaned_text=clean_data(removed_hyperlinks)
    # The final cleaned text is ready for word embedding
    return set(cleaned_text)

<h2><u>Iterate and assign scores</u></h2>
<p>We iterate through all the files and assign a positive/negative score to each tweet.</p>
<p><b>tweet_score</b> = negative_score - positive_score; negative_score and positive_score are both integers >= 0.</p>
<ol>
    <li>A &gt;0 overall_score means a negative sentiment, while a &lt;0 overall_score corresponds to a negative sentiment.</li>
    <li>A 0 overall_score reflects a neutral sentiment.</li>
</ol>
<p>At the end we will create a new files that will contain the scores of each tweet.</p>

<h4><u>Implementation</u></h4>
<p>Below we show the implementation of the functions that were needed to calculate a score to each tweet</p>

<h4>Get Hypernyms function</h4>
<p>The following function gets the hypernyms of each word in a tweet.</p>

In [40]:
def get_hypernyms(tweet):

    new_tweet = tweet

    # We create a counter to iterate through each word
    counter = 0
    # Now we iterate through each word to get the
    for w in tweet:
        synset = wordnet.synsets(w)

        # We check the length of the synset; we continue only if we get a response from wordnet.synsets
        if len(synset) > 0:
            # We get the hypernym
            hyper = synset[0].hypernyms()
            # If we get a result
            if len(hyper) > 0:
                first_lemma = hyper[0]
                lemmas = first_lemma.lemma_names()
                new_tweet[counter] = lemmas[0]

        # We increase the counter
        counter += 1

    return new_tweet

<h4>Get Tweet Scores function</h4>
<p>This function returns the score of the tweet</p>

In [41]:
def get_tweet_score(s):
    # We split the tweet by spaces to get each individual word.
    split_text = s.split()

    # We start by assigning 0 to the sentiment_score, this sentiment_score will consist of the sum of all
    # sentiment scores for each individual word.
    sentiment_score = 0  # sentiment_score

    # Here we specify the denominator that we will use to get the average
    # We will only take into account the words that have a score in wordnet.synsets
    syn_denominator = 0
    # We specify the overall score
    overall_score = 0

    # Now we replace each for in the tweet for its hypernym
    split_text = get_hypernyms(split_text)

    # Now we iterate through each word to get the
    for w in split_text:
        synset = wordnet.synsets(w)
        # We check the length of the synset; we continue only if we get a response from wordnet.synsets
        if len(synset) > 0:
            # synset[0].name() contains the id that we will use to reference the word later in senti_synset
            # We assign this value to the variable name
            name = synset[0].name()
            # Below we get the positive and negative scores for the word using its name as id
            breakdown = swn.senti_synset(name)
            # We get the negative and positive scores
            pos_score = breakdown.pos_score()
            neg_score = breakdown.neg_score()

            # Then we calculate the sentiment_score for this word and add it up to the scores of the previous words.
            sentiment_score += neg_score - pos_score  # The sentiment_score of all words in the tweet.

            # We increase the syn_denominator count + 1
            syn_denominator += 1

    # Now we will calculate the average using syn_denominator as denominator
    if syn_denominator != 0:
        # Now we calculate the average and assign it to the overall_score
        # The overall_score will be the mean of each score
        overall_score = sentiment_score / syn_denominator

    # We return the tweet's overall_score.
    return overall_score, split_text

<h4>Assign scores function</h4>
<p>The following is the implementation of assign scores phase.</p>

In [42]:
# Next we iterate through all the files and assign a score
def assign_scores(files_list):
    for file in files_list:
        # All files are txt files.
        f = open('Datasets/' + file + '.txt')
        # We read the lines of the txt file.
        lines = f.readlines()

        # We create the sentences dictionary. This will contain an array of sentences which each element will contain the
        # text and an overall score for the whole sentence.
        sentences = []

        # Counter for each sentence in the corpus, this is for later comparison with the file where we extracted the data.
        count = 0

        # For each line in the file we will proceed to
        for line in lines:
            # Data Extraction
            # We parse each fine of the file.
            parsed_json = (json.loads(line))

            # Discriminate to only take english sentences
            if parsed_json['lang'] == 'en':
                # We extract the text from the parsed line
                text = parsed_json['text']

                # Now we proceed to clean the text
                cleaned_set = Data_Cleaning(text)
                after_clean_text = " ".join(cleaned_set)

                # Now we iterate and we take the hypernyms of each word
                # We get the score for each sentence
                score, hypernyms = get_tweet_score(after_clean_text)

                # Dictionary creation
                # We create a dictionary for the sentence
                sentences.append({
                    'text': text,
                    'after_clean_text': after_clean_text,
                    'hypernyms': hypernyms,
                    'score': score
                })

            # We increase the counter
            count += 1

        # We create a new file for each hash tag file that we consulted.
        with open('Results/' + file + '_final_' + '.json', 'w') as outfile:
            # We finally dump the tweets + the overall_score in a json file.
            json.dump(sentences, outfile, indent=2)

        # End of the program's execution
        print("Total lines = " + str(count) + " we're processed.")
        print("Score assignation has finished successfully!")

<h4>Output</h4>
<p>We open the contents of the first file to check how the output looks like.</p>

In [43]:
# We input the list of files
files_list = ['tweets_fridaysforfuture_09-03-2020_test', 'tweets_climatechange_06-03-2020_test', 'tweets_savetheplanet_09-03-2020_test']
assign_scores(files_list)

# We load the url of the first file.
url = "Results/" + files_list[0] + ".json"

# We will open the file
with open(url, 'r') as json_file:
    data = json_file.read()

# Now we parse the file
parsed = json.loads(data)

# Pretty Printing JSON string back
print(json.dumps(parsed, indent = 2))

Total lines = 10 we're processed.
Score assignation has finished successfully!
Total lines = 10 we're processed.
Score assignation has finished successfully!
Total lines = 10 we're processed.
Score assignation has finished successfully!
[
  {
    "text": "RT @WantMyMoneyBac: @love4thegameAK Muslim Brothhood written goal is to infiltrate from within. Thanks Obama.\n\nBest 4 minutes you\u2019ll spend\u2026",
    "after_clean_text": "muslim obama infiltr wantmymoneybac best minut goal brothhood written within rt thank",
    "hypernyms": [
      "religious_person",
      "obama",
      "infiltr",
      "wantmymoneybac",
      "attempt",
      "minut",
      "content",
      "brothhood",
      "create_verbally",
      "within",
      "rt",
      "convey"
    ],
    "score": 0.0
  },
  {
    "text": "RT @RebeccaH2020: We chose @GretaThunberg because she stands up for what she believes\nChildren at a Yorkshire school chose the \"inspiring\"\u2026",
    "after_clean_text": "believ yorkshir sch

<h4>Run on all files.</h4>
<p>Below we will run the implemented code on all the test files.</p>

In [44]:
new_files_list = ['tweets_climatechange_06-03-2020', 'tweets_fridaysforfuture_09-03-2020', 'tweets_savetheplanet_09-03-2020']
assign_scores(new_files_list)

Total lines = 149399 we're processed.
Score assignation has finished successfully!
Total lines = 42315 we're processed.
Score assignation has finished successfully!
Total lines = 7385 we're processed.
Score assignation has finished successfully!


In [48]:
# We load the url of the first file.
url = "Results/" + new_files_list[0] + "_final_" + ".json"

# JSON file
f = open (url, "r")

# Reading from file
data = json.loads(f.read())

for i in range(4):
	print(str(data[i])+"\n")

{'text': 'RT @southpoleglobal: #IWD2020 this Sunday remind us that solutions to drive down emissions also need to support gender equality. Take a loo…', 'after_clean_text': 'take southpoleglob support equal also need solut sunday drive gender remind rt us emiss', 'hypernyms': ['income', 'southpoleglob', 'activity', 'person', 'also', 'condition', 'solut', 'rest_day', 'propulsion', 'grammatical_category', 'remind', 'rt', 'us', 'emiss'], 'score': -0.0125}

{'text': 'Sustainable ADA Signage – Top Six 2019 Trends: https://t.co/j8RxOw34qI #Signage #ADA #Greenbuilding #Architecture… https://t.co/jQqKgZLY2b', 'after_clean_text': 'six ada sustain greenbuild trend top signag', 'hypernyms': ['digit', 'enzyme', 'continue', 'greenbuild', 'direction', 'region', 'signag'], 'score': 0.0}

{'text': "Nori's @CleanEnergyGrl is the third (!) Norinaut to be on @zengineeringpod! Y'all really can't get enough, huh? Che… https://t.co/Hixb10A5ww", 'after_clean_text': 'norinaut nori zengineeringpod cant get eno